<a href="https://colab.research.google.com/github/alexanderDz/Web-Scraping-ML/blob/main/info_completa_Compus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aqui voy a sacar las siguientes variables para determinar como poder trabajar con ellas:
**Nombre**

**Precio**

**Rating**

**Numero de reviews**

**Comentarios**

**Cantidad de ventas (aprox) en 60 dias**

**Ubicacion**



In [ ]:
#Instalacion de librerias
!pip install beautifulsoup4 
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#%% librerias
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
#%%
def recorre_paginas(n):
    
    if n == 1:
        url = 'https://listado.mercadolibre.com.co/computacion/portatiles-accesorios/portatiles/portatiles_NoIndex_True'
    elif n > 1:
        x = str(5*(n - 1)) + '1'
        url = f'https://listado.mercadolibre.com.co/computacion/portatiles-accesorios/portatiles/portatiles_Desde_{x}_NoIndex_True'
        
    return url

In [ ]:
#%%
def solicitud(url):
    pagina = requests.get(url)
    pagina.status_code
    return BeautifulSoup(pagina.content,'html.parser')

#%%

In [ ]:
#%%
def extraer_info(url):
    soup = solicitud(url)#5
    
    if soup.find('h1',class_='ui-pdp-title') is not None:
        titulo = soup.find('h1',class_='ui-pdp-title').get_text()
    else:
        titulo = ""
    
    if soup.find('span',class_='andes-money-amount__fraction') is not None:    
        precio = soup.find('span',class_='andes-money-amount__fraction').get_text()
    else:
        precio = 0
    
    if soup.find('p',class_='ui-review-capability__rating__average ui-review-capability__rating__average--desktop') is not None:
        rating = soup.find('p',class_='ui-review-capability__rating__average ui-review-capability__rating__average--desktop').get_text()
    else:
        rating = 0

    if soup.find('strong',class_='ui-pdp-seller__sales-description') is not None:
        ventas_60 = soup.find('strong',class_='ui-pdp-seller__sales-description').get_text()
    else:
        ventas_60 = 0    

    ########
    if soup.find('a',class_='ui-pdp-media__action ui-box-component__action') is not None:
        link_info_vendedor = soup.find('a',class_='ui-pdp-media__action ui-box-component__action').get('href')
        soup2 = solicitud(link_info_vendedor)

        if soup2.find('p',class_='location-subtitle') is not None:
          ubicacion = soup2.find('p',class_='location-subtitle').get_text()
        else:
          ubicacion = ""
    else:
      ubicacion = ""
        
    return titulo, precio, rating, ubicacion, ventas_60

In [ ]:
#%%
def extrae_pag(links):
    titulo, precio, rating, ubicacion, ventas_60 = [],[],[],[],[]
    for i in range(0,50):
      try:
          a, b, c, d, e  = extraer_info(links[i])
      except Exception as e:
        print(f"Error al extraer información del enlace {links[i]}: {str(e)}")
        continue
      titulo.append(a), precio.append(b), rating.append(c), ubicacion.append(d), ventas_60.append(e)
    return {'Titulo':titulo, 'Precio':precio, 'Rating':rating, 'Ubicacion':ubicacion, 'Ventas 60 dias':ventas_60}  

In [ ]:
#%%
def paginas(n):
    box = solicitud(recorre_paginas(n)).find_all('a', class_='ui-search-item__group__element shops__items-group-details ui-search-link', limit = 50)    
    links = [i.get('href') for i in box]
    datos = extrae_pag(links)
    return pd.DataFrame(datos)


In [ ]:
dfs = []
for n in range (1,11):
  try:  
        df_parcial = paginas(n)      
        dfs.append(df_parcial)
  except:
        print(f"Error en la iteración {n}")
df = pd.concat(dfs,ignore_index=True)
#print(df)
#df = pd.concat([paginas(n) for n in range (1,2)],ignore_index=True)


Error en la iteración 4


In [ ]:
print(df)

                                                Titulo     Precio Rating  \
0    Laptop HP 240 G7 gris 14", Intel Celeron N4020...  1.180.000    4.7   
1    Laptop Asus M515DA gris 15.6", AMD Ryzen 5 350...  2.473.572    4.7   
2    Laptop gamer Asus TUF Gaming FX506LH negra 15....  3.545.534    5.0   
3    Laptop Dell Inspiron 3505 gris 15.6", AMD Ryze...  2.036.000    4.6   
4    Laptop Asus Vivobook Pro 15 OLED quiet blue 15...  2.489.999    4.8   
..                                                 ...        ...    ...   
445  Asus Tuf Gaming F15 (2022) I7 12700h 14c 16gb ...  8.590.000      0   
446  Msi Gf65 Thin I5 10500h Rtx 3060 512gb 144hz P...  5.690.000    5.0   
447  Portatil Gamer Aorus 15 I7 12700h Rtx 3070 Ti ...  8.990.000      0   
448  Lenovo Legión 5 Pro 16  165hz Ryzen 7 5800h Rt...  6.890.000      0   
449  Portatil Gamer Msi Gl 66 I7 11800h Rtx 3070 16...  7.690.000      0   

                   Ubicacion Ventas 60 dias  
0    Chapinero, Bogotá D.C..           +1

In [ ]:
df.to_csv('datos_portatiles_1.csv', index=False)

# Descargar archivo CSV a la computadora local
from google.colab import files
files.download('datos_portatiles_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.to_excel("datos_portatiles.xlsx", index=False)
files.download("datos_portatiles.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>